In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWhwah8edQlCkexUd3e1zua1fZNkREOvLXYGGmTvRmc9mKbiGPaNcJY



Successfully saved authorization token.


In [17]:
import ee
import geemap

Map = geemap.Map()

geometry = ee.Geometry.Polygon(
    [[[25.28718028631623, -10.457488732614895],
      [25.28718028631623, -10.854273006505112],
      [26.285562854675604, -10.854273006505112],
      [26.285562854675604, -10.457488732614895]]], None, False)
Map.centerObject(geometry, 10);
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
mines = ee.FeatureCollection("users/EmilyNason/cod_mines_curated_all_opendata_p_ipis")
gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

before = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-06-01', '2020-01-01')) \
  .filter(ee.Filter.intersects('.geo', geometry)) \
  .median().clip(geometry)
after = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2021-06-01', '2022-01-01')) \
  .filter(ee.Filter.intersects('.geo', geometry)) \
  .median().clip(geometry)

Map.addLayer(before, rgbVis, 'before')
Map.addLayer(after, rgbVis, 'after')

def addIndices(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
  return image.addBands(ndvi)

beforeIndices = addIndices(before).clip(geometry)
afterIndices = addIndices(after).clip(geometry)

ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}

Map.addLayer(beforeIndices.select('ndvi'), ndviVis, 'NDVI 2019', False)
Map.addLayer(afterIndices.select('ndvi'), ndviVis, 'NDVI 2021', False)

#low ndvi -> possible existing mining
# 2019
mining2019 = beforeIndices.select('ndvi').lt(.15).rename('mining').selfMask()
permanentWater = gsw.select('seasonality').gte(5).clip(geometry)
mining2019 = mining2019.where(permanentWater,0).selfMask()
cpThreshold = 8
connections = mining2019.connectedPixelCount(25)
mining2019 = mining2019.updateMask(connections.gt(cpThreshold))
Map.addLayer(mining2019, {'min':0, 'max':1, 'palette': ['yellow']}, 'Possible Mining Area 2019')
# 2021
mining2021 = afterIndices.select('ndvi').lt(.15).rename('mining').selfMask()
permanentWater = gsw.select('seasonality').gte(5).clip(geometry)
mining2021 = mining2021.where(permanentWater,0).selfMask()
cpThreshold = 8
connections2 = mining2021.connectedPixelCount(25)
mining2021 = mining2021.updateMask(connections2.gt(cpThreshold))
Map.addLayer(mining2021, {'min':0, 'max':1, 'palette': ['orange']}, 'Possible Mining Area 2021')

#ndvi loss -> possible new mining
difference = beforeIndices.select('ndvi').divide(afterIndices.select('ndvi'))
threshold = 1.5
newMining = difference.gt(threshold).rename('newMining').selfMask()
newMining = newMining.where(permanentWater,0).selfMask()
connections3 = newMining.connectedPixelCount(25)
newMining = newMining.updateMask(connections3.gt(cpThreshold))
Map.addLayer(newMining, {'min':0, 'max':1, 'palette': ['red']}, 'Possible New Mining Area')

Map.addLayer(mines, {'color': 'blue'}, 'mines')

# LEGEND
legend_dict = {
    'Possible Mining Areas 2019': ('FFFF00'),
    'Possible Mining Areas 2021': ('FFA500'),
    'Possible Increased Activity': ('FF0000'),
    'Known Active Mine Locations': ('0000FF'),
    'NDVI Level': ('008000'),
}
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')
Map


Map(center=[-10.655969510994254, 25.78637157049617], controls=(WidgetControl(options=['position', 'transparent…